# Dataset Creation

In [1]:
import os
from scipy import misc
import numpy as np
from matplotlib import pyplot as plt
from keras.utils.data_utils import get_file
from six.moves import cPickle
import sys
from keras.utils import np_utils 
import cPickle as pickle
from sklearn.cross_validation import train_test_split

/Users/peterhirt/anaconda/envs/theano/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [4]:
# define the current work directory
cwd=os.getcwd()
cwd

'/Users/peterhirt/datascience/ild-cnn/notebooks'

In [5]:
one_folder_up = os.path.dirname(os.getcwd())
print one_folder_up

/Users/peterhirt/datascience/ild-cnn


In [7]:
# create path to patch directory
patch_dir = os.path.join(one_folder_up, 'patches')
print patch_dir

/Users/peterhirt/datascience/ild-cnn/patches


In [8]:
# list all directories under patch directory. They are representing the categories
category_list = (os.listdir(patch_dir))
print 'taking out the item : ', category_list.pop(0)

taking out the item :  .DS_Store


In [9]:
# print what we have as categories
category_list

['consolidation',
 'fibrosis',
 'ground_glass',
 'healthy',
 'micronodules',
 'reticulation']

In [12]:
# creating variables
# list for the merged pixel data
dataset_list = []
# list of the label data
label_list = []



# go through all categories
# 
for category in category_list:
    
    category_dir = os.path.join(patch_dir, category)
    print  'the path into the categories is: ', category_dir
    
    sub_categories_dir_list = (os.listdir(category_dir))
    print 'the sub categories are : ', sub_categories_dir_list
    sub_categories_dir_list.pop(0)
        
    for subCategory in sub_categories_dir_list:
            
        subCategory_dir = os.path.join(category_dir, subCategory)
        print  'the path into the sub categories is: '
        print subCategory_dir
        
        image_files = (os.listdir(subCategory_dir))
            
        for file in image_files:
                
            if file.find('.bmp') > 0:
                
                # load the .bmp file into array       
                image = misc.imread(os.path.join(subCategory_dir,file), flatten= 0)
                # append the array to the dataset list
                dataset_list.append(image)
                
                # created rotated copies of images
                image90 = np.rot90(image)
                dataset_list.append(image90)
                
                image180 = np.rot90(image90)
                dataset_list.append(image180)
                
                image270 = np.rot90(image180)
                dataset_list.append(image270)
                
                if category == 'consolidation':    
                    label_list.append(0)
                    # append also lables for rotated images
                    label_list.append(0)
                    label_list.append(0)
                    label_list.append(0)
                    
                if category == 'fibrosis':    
                    label_list.append(1)
                    # append also lables for rotated images
                    label_list.append(1)
                    label_list.append(1)
                    label_list.append(1)
                    
                if category == 'ground_glass':    
                    label_list.append(2)
                    # append also lables for rotated images
                    label_list.append(2)
                    label_list.append(2)
                    label_list.append(2)
                    
                if category == 'healthy':    
                    label_list.append(3)
                    # append also lables for rotated images
                    label_list.append(3)
                    label_list.append(3)
                    label_list.append(3)
                    
                if category == 'micronodules':    
                    label_list.append(4)
                    # append also lables for rotated images
                    label_list.append(4)
                    label_list.append(4)
                    label_list.append(4)
                    
                if category == 'reticulation':    
                    label_list.append(5)
                    # append also lables for rotated images
                    label_list.append(5)
                    label_list.append(5)
                    label_list.append(5)
                                 
# transform dataset list into numpy array                   
dataset = np.array(dataset_list)

# use only one of the 3 color channels as greyscale info
X = dataset[:,:, :,1]

print 'dataset shape is now: ', X.shape

# 
y = np.array(label_list)
# sampling item 22
y[22]


            

the path into the categories is:  /Users/peterhirt/datascience/ild-cnn/patches/consolidation
the sub categories are :  ['.DS_Store', 'consolidation_apical', 'consolidation_diffuse', 'consolidation_non-relevant', 'consolidation_peripheral_sub_pleural']
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/consolidation_apical
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/consolidation_diffuse
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/consolidation_non-relevant
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/consolidation_peripheral_sub_pleural
the path into the categories is:  /Users/peterhirt/datascience/ild-cnn/patches/fibrosis
the sub categories are :  ['.DS_Store', 'fibrosis_apical', 'fibrosis_basal', 'fibrosis_diffuse', 'fibrosis_non-relevant', 'fibrosis_perihilar', 'fibrosis_peripheral_

0

In [13]:
print X.shape
print y.shape

(25068, 32, 32)
(25068,)


In [14]:
# use only x thousand items out of the total
d1 = X.shape[0]
d2 = d1 % 1000

d3 = d1 - d2
d3
X = X[0:d3]
y = y[0:d3]

In [15]:
print X.shape
print y.shape

(25000, 32, 32)
(25000,)


In [16]:
X_train, X_intermediate, y_train, y_intermediate = train_test_split(X, y, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.5, random_state=42)
print X_train.shape
print X_val.shape
print X_test.shape
print y_train.shape
print y_test.shape
print y_val.shape
    

(12500, 32, 32)
(6250, 32, 32)
(6250, 32, 32)
(12500,)
(6250,)
(6250,)


In [17]:
# save the dataset and label set into serial formatted pkl 

pickle.dump(X_train, open( "../pickle/X_train.pkl", "wb" ))
pickle.dump(X_test, open( "../pickle/X_test.pkl", "wb" ))
pickle.dump(X_val, open( "../pickle/X_val.pkl", "wb" ))
pickle.dump(y_train, open( "../pickle/y_train.pkl", "wb" ))
pickle.dump(y_test, open( "../pickle/y_test.pkl", "wb" ))
pickle.dump(y_val, open( "../pickle/y_val.pkl", "wb" ))

In [18]:
# testing if pickls was working fine
recuperated_X_train = pickle.load( open( "../pickle/X_train.pkl", "rb" ) )

In [19]:
recuperated_X_train

array([[[107,  75,  60, ...,  72,  92,  71],
        [ 75,  80,  69, ...,  76,  79,  68],
        [ 65,  71,  68, ...,  74,  81,  71],
        ..., 
        [ 70,  76,  66, ..., 147,  80,  53],
        [ 75,  76,  50, ..., 197, 174,  77],
        [ 80,  68,  47, ..., 150, 223, 151]],

       [[ 99, 103, 144, ..., 201, 222, 211],
        [ 63,  14,  50, ..., 217, 252, 220],
        [ 24,  29,  43, ..., 221, 219, 231],
        ..., 
        [140, 163, 103, ..., 150, 126,  50],
        [ 77, 100, 107, ..., 169, 130,  21],
        [ 74,  64,  73, ...,  81,  91,  57]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       ..., 
       [[193, 180,  43, ..., 198, 186, 218],
        [154, 124,  85, ..., 228, 229, 254],
        [168, 207, 182, ..., 255, 255, 